In [1]:
!gdown --id "1-23aNm7j0bnycvyd_OaQfofVYPTewgOI"   # abstract vectors
!gdown --id "1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX"   # TitlesAbstractsEmbedIds
!wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
!tar -xvf 'scibert_scivocab_uncased.tar'

Downloading...
From: https://drive.google.com/uc?id=1-23aNm7j0bnycvyd_OaQfofVYPTewgOI
To: /content/AbstractSimVectors.npy
2.59GB [00:21, 121MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX
To: /content/TitlesAbstractsEmbedIds.json.gzip
397MB [00:02, 139MB/s] 
--2019-12-29 13:52:18--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.212.32
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.212.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 421.96M  29.4MB/s    in 10s     

2019-12-29 13:52:29 (40.4 MB/s) - ‘scibert_scivocab_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert

In [0]:
!pip install transformers --quiet

In [1]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
from transformers import BertTokenizer
import pandas as pd
from pprint import pprint

print('TensorFlow:', tf.__version__)

TensorFlow 2.x selected.
TensorFlow: 2.1.0-rc1


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    print('Failed initializing TPU, cannot proceed')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: 10.110.79.146:8470


INFO:tensorflow:Initializing the TPU system: 10.110.79.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
class Index:
    def __init__(self, vectors, worker):
        self.vectors = vectors
        self.worker = worker
        print('Building index with {} vectors on {}'.format(
            vectors.shape[0], worker))

    @tf.function
    def search(self, query_vector, top_k=20):
        with tf.device(self.worker):
            dot_product = tf.reduce_sum(tf.multiply(
                self.vectors, query_vector), axis=1)
            distances = 1 - dot_product
            sorted_indices = tf.argsort(distances)
            nearest_distances = tf.gather(distances, sorted_indices)
            return nearest_distances[:top_k], sorted_indices[:top_k]
        # ToDo: Add search using L2 distance


class TPUIndex:
    def __init__(self, num_tpu_cores=8):
        self.workers = ['/job:worker/replica:0/task:0/device:TPU:' + str(i)
                        for i in range(num_tpu_cores)]
        self.indices = [None] * num_tpu_cores
        self.normalized_vectors = False

    def create_index(self, vectors, normalize=True):
        self.vectors = vectors
        self.normalized_vectors = normalize

        drop = self.vectors.shape[0] % len(self.workers)
        self.vecs_per_index = self.vectors.shape[0] // len(self.workers)
        self.vectors = self.vectors[:-drop]
        self.vectors = np.split(self.vectors, len(self.workers), axis=0)

        for i in range(len(self.workers)):
            worker = self.workers[i]
            with tf.device(worker):
                vecs = self.vectors[i]
                if self.normalized_vectors:
                    vecs = tf.math.l2_normalize(vecs, axis=1)
                self.indices[i] = Index(vecs, worker)

    def search(self, xq, distance_metric='cosine', top_k=10):
        dims = xq.shape

        assert len(dims) == 2, \
            '''Expected query_vector to have 2 dimesions but
               found {}'''.format(len(dims))

        assert dims[0] == 1, \
            '''Expected query_vector to have shape == [1, None]
               but got'''.format(dims)

        if distance_metric == 'cosine':
            assert self.normalized_vectors, \
                '''Currently only normalized vectors are supported
                   for searching with cosine distances'''

        Dx, Ix = [], []
        for i in range(len(self.workers)):
            print('Search running on {}'.format(self.indices[i].worker))
            d, idx = self.indices[i].search(xq, top_k)
            Dx.extend(d.numpy())
            Ix.extend(i * self.vecs_per_index + idx.numpy())

        # ToDo: Dont sort again, merge the already sorted distance arrays
        id_sorted = np.argsort(Dx)[:top_k]
        Dx = np.array(Dx)[id_sorted]
        Ix = np.array(Ix)[id_sorted]
        return Dx, Ix

In [4]:
abstract_embeddings = np.load('AbstractSimVectors.npy')
abstract_embeddings.shape

(1262996, 512)

In [5]:
index = TPUIndex(num_tpu_cores=strategy.num_replicas_in_sync)
index.create_index(abstract_embeddings)

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
with strategy.scope():
  model = tf.saved_model.load('gs://tfworld/saved_models')

In [0]:
tokenizer = BertTokenizer(vocab_file='scibert_scivocab_uncased/vocab.txt')

df = pd.read_json('TitlesAbstractsEmbedIds.json.gzip', compression = 'gzip')
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Abstract = pd.Series(df['paperAbstract'].values,index=df['EmbeddingID']).to_dict()

In [8]:
embed_id = 567
title = embed2Title[embed_id]
abstract = embed2Abstract[embed_id]

abstract_encoded = tokenizer.encode(abstract, max_length=512, pad_to_max_length=True)
abstract_encoded = tf.constant(abstract_encoded, dtype=tf.int32)[None, :]
print('Title : ')
pprint(title)
print('\nAbstract : ')
pprint(abstract)

Title : 
'DVCC based current-mode first-order all-pass filter and its application'

Abstract : 
('In this paper, a new realisation of DVCC based current-mode all-pass filter '
 'circuit is proposed. This circuit employs only one active element and three '
 'passive elements of which two are grounded. Using the proposed filter '
 'topology a new quadrature oscillator is presented as an application. SPICE '
 'simulations verify the theoretical results.')


In [10]:
s = time()
bert_output = model(abstract_encoded)
xq = tf.nn.l2_normalize(bert_output, axis=1)
e_p = time()

aD, aI = index.search(xq, top_k=5)
e_s = time()
print('\n'*2)
print('Prediction time  :', np.round(e_p-s, 3), 'secs')
print('Search time      :', np.round(e_s-e_p, 3), 'secs')
print('Total time       :', np.round(e_s - s, 3), 'secs')

print('\n'*4)
print('*'*80)
for i in range(len(aI)):
  print('Abstract : ')
  pprint(embed2Abstract[aI[i]])
  print('*'*80)
print('\nNeighbours       :', aI )
print('Distances        :', np.round(aD, 4))

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7



Prediction time  : 0.002 secs
Search time      : 0.752 secs
Total time       : 0.754 secs





********************************************************************************
Abstract : 
('In this paper, a new realisation of DVCC based current-mode all-pass filter '
 'circuit is proposed. This circuit employs only one active element and three '
 'passive elements of which two are grounded. Using the proposed filter '
 'topology a new quadrature oscillator is presented as an application. SPICE '
 'simulations ve